# Logistic Regression with manual Gradient Descent

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

X1 = np.linspace(-4, 4) # for plotting

n = 1000
q = 2
X = np.random.normal(size = n * q).reshape((n, q))
beta = [1.0, 2.0]
p = 1 / (1 + np.exp(-np.dot(X, beta)))
y = np.random.binomial(1, p, size = n)

def plot_sim(plot_beta_hat=True):
  plt.clf()
  plt.scatter(X[:, 0], X[:, 1], c = y)
  plt.plot(X1, -X1 * beta[0]/beta[1], linestyle = '--', color = 'red')
  if plot_beta_hat:
    plt.plot(X1, -X1 * beta_hat[0]/beta_hat[1], linestyle = '--')
  plt.xlabel('X1')
  plt.ylabel('X2')
  if plot_beta_hat:
    title = 'Guess: %.2f * X1 + %.2f * X2 = 0' % (beta_hat[0], beta_hat[1])
  else:
    title = 'Ideal: 1 * X1 + 2 * X2 = 0'
  plt.title(title)
  plt.show()

In [ ]:
plot_sim(False)

In [ ]:
# initial guess for beta

beta_hat = np.ones(q) # [1, 1]

plot_sim()

In [ ]:
# let's do 1 iteration

alpha = 0.01

p_hat = 1 / (1 + np.exp(-np.dot(X, beta_hat)))
grad = -np.dot(X.T, (y - p_hat))
beta_hat = beta_hat - alpha * grad

plot_sim()

In [ ]:
# let's do 10 more iterations

for i in range(10):
  p_hat = 1 / (1 + np.exp(-np.dot(X, beta_hat)))
  grad = -np.dot(X.T, (y - p_hat))
  beta_hat = beta_hat - alpha * grad

plot_sim()

In [ ]:
# let's see NLL -l(beta) through iterations

alpha = 0.001
beta_hat = np.array([-2.5, -2.5])
betas = [beta_hat]
ls = []
for i in range(50):
  p_hat = 1 / (1 + np.exp(-np.dot(X, beta_hat)))
  nll = -np.sum(y * np.log(p_hat) + (1 - y) * np.log(1 - p_hat))
  ls.append(nll)
  grad = -np.dot(X.T, (y - p_hat))
  beta_hat = beta_hat - alpha * grad
  betas.append(beta_hat)

plt.plot(range(50), ls)
plt.xlabel("Iteration")
plt.ylabel(r"$-l(\beta)$")
plt.show()

In [ ]:
# Even fancier, visualize the actual gradient descent in the beta space:

betas_arr = np.array(betas)
m = 10
beta1 = np.linspace(-3.0, 3.0, m)
beta2 = np.linspace(-3.0, 3.0, m)
B1, B2 = np.meshgrid(beta1, beta2)
L = np.zeros((m, m))
for i in range(m):
  for j in range(m):
    beta_hat = np.array([beta1[i], beta2[j]])
    p_hat = 1 / (1 + np.exp(-np.dot(X, beta_hat)))
    L[i, j] = -np.sum(y * np.log(p_hat) + (1 - y) * np.log(1 - p_hat))
fig, ax = plt.subplots(1,1)
cp = ax.contourf(B1, B2, L)
cb = fig.colorbar(cp)
ax.set_title(r'$-l(\beta)$ gradient descent')
ax.set_xlabel(r'$\beta_1$')
ax.set_ylabel(r'$\beta_2$')
ax.plot(betas_arr[:, 0], betas_arr[:, 1], marker='x', color='white')
ax.plot([beta[0]], [beta[1]], marker='x', color='red', markersize=20, markeredgewidth=5)
plt.show()

# Logistic Regression as NN

In [ ]:
#| echo: true
#| code-line-numbers: "|1-6|8-12|14-15|17-21|"

def forward(X, y, beta_hat):
  z = np.dot(X, beta_hat)
  p_hat = 1 / (1 + np.exp(-z))
  l = y * np.log(p_hat) + (1 - y) * np.log(1 - p_hat)
  nll = -np.sum(l)
  return p_hat, nll

def backward(X, y, p_hat):
  dldz = y - p_hat
  dzdb = X.T
  grad = -np.dot(dzdb, dldz)
  return grad

def gradient_descent(alpha, beta_hat, grad):
  return beta_hat - alpha * grad

def optimize(X, y, alpha, beta_hat):
  p_hat, l = forward(X, y, beta_hat)
  grad = backward(X, y, p_hat)
  beta_hat = gradient_descent(alpha, beta_hat, grad)
  return l, beta_hat

In [ ]:
#| echo: true

def lr_nn(X, y, epochs):
  beta_hat = np.array([-2.5, -2.5])
  alpha = 0.001
  for i in range(epochs):
    l, beta_hat = optimize(X, y, alpha, beta_hat)
  return l, beta_hat

In [ ]:
#| echo: true

def lr_nn(X, y, epochs):
  beta_hat = np.random.rand(X.shape[1])
  alpha = 0.001
  batch_size = 100
  n = X.shape[0]
  steps = int(n / batch_size)
  for i in range(epochs):
    print('epoch %d:' % i)
    permute = np.random.permutation(n)
    X_perm = X[permute, :]
    y_perm = y[permute]
    for j in range(steps):
      start = j * batch_size
      l, beta_hat = optimize(X_perm[start:start + batch_size, :],
                            y_perm[start:start + batch_size],
                            alpha, beta_hat)
      print('Trained on %d/%d, loss = %d' % (start + batch_size, n, l))
  return l, beta_hat

l, beta_hat = lr_nn(X, y, 50)

# Logistic Regression as NN in `Keras`

In [ ]:
#| echo: true
#| code-line-numbers: "|1|2|3|5|6-7|8|9|10|"

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

model = Sequential()
model.add(Dense(1, input_shape=(X.shape[1], ),
  activation='sigmoid', use_bias=False))
sgd = SGD(learning_rate=0.1)
model.compile(loss='binary_crossentropy', optimizer=sgd)
model.fit(X, y, batch_size=100, epochs=50)

In [ ]:
# See that it makes sense:
beta_hat = model.get_weights() # Note Keras gives a list of weights!
beta_hat

In [ ]:
pred = model.predict(X, verbose=0)
pred[:3]

In [ ]:
pred_manual = 1/(1+np.exp(-np.dot(X, beta_hat[0])))
pred_manual[:3]

# Adding C nuerons for C classes

In [ ]:
#| echo: true
#| code-line-numbers: "|3|5-6|8|"

from tensorflow.keras.utils import to_categorical

y_categorical = to_categorical(y)
model = Sequential()
model.add(Dense(2, input_shape=(X.shape[1], ),
  activation='softmax', use_bias=False))
sgd = SGD(learning_rate=0.1)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
model.fit(X, y_categorical, batch_size=100, epochs=50)

In [ ]:
# See that it makes sense:
W = model.get_weights()
W

In [ ]:
pred = model.predict(X, verbose=0)
pred[:3]

In [ ]:
Z = np.dot(X, W[0])
Z_exp = np.exp(Z)
Z_exp_sum = Z_exp.sum(axis=1)[:, None]
pred_manual = Z_exp / Z_exp_sum
pred_manual[:3]

# Adding Hidden Layers

In [ ]:
#| eval: true

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

In [ ]:
#| eval: true
#| echo: true

model = Sequential()
model.add(Dense(4, input_shape=(X.shape[1], ), activation='sigmoid'))
model.add(Dense(2, activation='softmax'))
sgd = SGD(learning_rate=0.1)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

In [ ]:
model.fit(X, y_categorical, batch_size=100, epochs=50)

In [ ]:
#| eval: true
#| echo: true

model.summary()

In [ ]:
# See that it makes sense:
pred = model.predict(X, verbose=0)

pred[:3]

In [ ]:
W1, b1, W2, b2 = model.get_weights()

W1.shape # (2, 4)
b1.shape # (4,)
W2.shape # (4, 2)
b2.shape # (2,)

W1 = np.vstack([b1, W1])
W2 = np.vstack([b2, W2])

W1.shape # (3, 4)
W2.shape # (5, 2)

# Get X ready with an intercept column
Xb = np.hstack((np.ones(n).reshape((n, 1)), X))
Xb.shape # (1000, 3)

In [ ]:
Z = 1/(1 + np.exp(-np.dot(Xb, W1)))
Zb = np.hstack((np.ones(n).reshape((n, 1)), Z))
Z2_exp = np.exp(np.dot(Zb, W2))
Z2_exp_sum = Z2_exp.sum(axis=1)[:, None]
pred_manual = Z2_exp / Z2_exp_sum

pred_manual[:3]

# Malaria classification with Keras

In [ ]:
#| echo: true
#| eval: true

import tensorflow_datasets as tfds
from skimage.transform import resize

malaria, info = tfds.load('malaria', split='train', with_info=True)
fig = tfds.show_examples(malaria, info)

In [ ]:
#| eval: true
#| echo: true
#| code-line-numbers: "|16-17|"

from sklearn.model_selection import train_test_split

images = []
labels = []
for example in tfds.as_numpy(malaria):
  images.append(resize(example['image'], (100, 100)))
  labels.append(example['label'])
  if len(images) == 2500:
    break
  
X = np.array(images)
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

X_train = X_train.flatten().reshape((X_train.shape[0], -1))
X_test = X_test.flatten().reshape((X_test.shape[0], -1))

print(X_train.shape)
print(X_test.shape)

In [ ]:
#| eval: true
#| echo: true

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='none', max_iter=1000, random_state=42)
lr = lr.fit(X_train, y_train)

test_acc = lr.score(X_test, y_test)
print(f'Test accuracy for LR: {test_acc:.3f}')

In [ ]:
#| eval: true
#| echo: true

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(300, input_shape=(30000,), activation='relu', name='my_dense_layer'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
#| eval: true
#| echo: true

model.summary()

In [ ]:
#| eval: true
#| echo: true

model.layers

In [ ]:
#| eval: true
#| echo: true

model.layers[0].name

In [ ]:
#| eval: true
#| echo: true

W1, b1 = model.get_layer('my_dense_layer').get_weights()

print(W1.shape)
W1

In [ ]:
#| eval: true
#| echo: true

model.compile(loss="binary_crossentropy",
  optimizer="adam",
  metrics=["accuracy"])

In [ ]:
#| echo: true
#| eval: true

from tensorflow.keras.callbacks import EarlyStopping

callbacks = [EarlyStopping(monitor='val_loss', patience=5,
  restore_best_weights=True)]

history = model.fit(X_train, y_train,
  batch_size=100, epochs=50,
  validation_split=0.1, callbacks=callbacks, verbose=0)

In [ ]:
#| eval: true
#| echo: true

import pandas as pd

pd.DataFrame(history.history).plot(figsize=(10, 6))
plt.grid(True)
plt.show()

In [ ]:
#| eval: true
#| echo: true

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=False)
print(f'Test accuracy for NN: {test_acc:.3f}')

In [ ]:
#| eval: true
#| echo: true
#| code-line-numbers: "|3|5|"

from sklearn.metrics import confusion_matrix

y_pred = (model.predict(X_test, verbose=0) > 0.5).astype(int).reshape(y_test.shape)
pd.DataFrame(
  confusion_matrix(y_test, y_pred), 
  index=['true:yes', 'true:no'], 
  columns=['pred:yes', 'pred:no']
)

In [ ]:
!pip install scikeras

In [ ]:
#| echo: true
#| code-line-numbers: "|5|"

from tensorflow.keras.layers import InputLayer
from tensorflow.keras.optimizers import SGD
from scikeras.wrappers import KerasClassifier

def malaria_model(n_hidden, n_neurons, lrt):
  model = Sequential()
  model.add(InputLayer(input_shape=(30000, )))
  for layer in range(n_hidden):
    model.add(Dense(n_neurons, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss="binary_crossentropy",
    optimizer=SGD(learning_rate=lrt),
    metrics=["accuracy"])
  return model

keras_clf = KerasClassifier(model=malaria_model, n_hidden=1, n_neurons=30, lrt=3e-3)

In [ ]:
#| echo: true
#| code-line-numbers: "|4-8|10-13|"

from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

params = {
  'n_hidden': [0, 1, 2, 3],
  'n_neurons': np.arange(1, 100),
  'lrt': reciprocal(3e-4, 3e-2)
}

rnd_search_cv = RandomizedSearchCV(keras_clf, params, cv=5,
  n_iter=10)
rnd_search_cv.fit(X_train, y_train, epochs=50,
  validation_split=0.1, callbacks=callbacks)

print(f'Best test accuracy: {rnd_search_cv.best_score_:.2f}')
print(f'Best params: {rnd_search_cv.best_params_}')

In [ ]:
#| eval: true
#| echo: true

model.save('malaria.h5')

In [ ]:
#| eval: true
#| echo: true

model = keras.models.load_model('malaria.h5')
model.predict(X_test[:3], verbose=0)